In [35]:
import nn_helpers
import torch
from torch import nn
import pandas as pd
import numpy as np
import data_preprocessing
from sklearn.preprocessing import MinMaxScaler

In [17]:
seq_len = 10

def net_to_str(num_hiddens, lr, num_out_neutrons):
  return f'{num_hiddens}-{lr}-{num_out_neutrons}'

def path_to_net(num_hiddens, lr, num_out_neutrons, is_gru, is_cut):
  return f'models/{"/cut/" if is_cut else ""}{"gru" if is_gru else "lstm"}/{num_hiddens}-{lr}-{num_out_neutrons}.pt'


def net_from_row(num_hiddens, lr, num_out_neutrons, is_gru):
  num_out_neutrons = None if num_out_neutrons == 'None' else [num_hiddens]
  return nn_helpers.RNNModel(seq_len, num_hiddens, 2, num_hidden_neutrons=num_out_neutrons, recur_ctor=nn.GRU if is_gru else nn.LSTM)


In [4]:
lstm_hist = pd.read_csv('models/lstm/hists.csv')
gru_hist = pd.read_csv('models/gru/hists.csv')
lstm_cut_hist = pd.read_csv('models/cut/lstm/hists.csv')
gru_cut_hist = pd.read_csv('models/cut/gru/hists.csv')

In [7]:
main_df = data_preprocessing.get_data('BTC-USD')

In [38]:
scaler = MinMaxScaler()
scaler = scaler.fit(np.array(main_df['Close'].tolist()).reshape(-1, 1))

In [40]:
X_test = main_df[(main_df['Date'] < '2022-05-10') & (main_df['Date'] >= '2022-05-01')]
print(X_test)
y_test = main_df[main_df['Date'] == '2022-05-10']
print(y_test)

X_test = scaler.transform(np.array(X_test['Close']).reshape(-1, 1)).reshape(1, -1, 1)
y_test = scaler.transform(np.array(y_test['Close']).reshape(-1, 1))

X_test, y_test

           Date          Open          High           Low         Close  \
2783 2022-05-01  37713.265625  38627.859375  37585.789063  38469.093750   
2784 2022-05-02  38472.187500  39074.972656  38156.562500  38529.328125   
2785 2022-05-03  38528.109375  38629.996094  37585.621094  37750.453125   
2786 2022-05-04  37748.011719  39902.949219  37732.058594  39698.371094   
2787 2022-05-05  39695.746094  39789.281250  35856.515625  36575.140625   
2788 2022-05-06  36573.183594  36624.359375  35482.132813  36040.921875   
2789 2022-05-07  36042.503906  36129.925781  34940.824219  35501.953125   
2790 2022-05-08  35502.941406  35502.941406  33878.964844  34059.265625   
2791 2022-05-09  34060.015625  34222.074219  30296.953125  30296.953125   

         Adj Close       Volume  
2783  38469.093750  27002760110  
2784  38529.328125  32922642426  
2785  37750.453125  27326943244  
2786  39698.371094  36754404490  
2787  36575.140625  43106256317  
2788  36040.921875  37795577489  
2789  35501

(array([[[0.56821064],
         [0.56910448],
         [0.55754653],
         [0.58645223],
         [0.54010575],
         [0.53217833],
         [0.52418042],
         [0.50277198],
         [0.44694198]]]),
 array([[0.4577146]]))

In [41]:
hists = {}
X_test = torch.tensor(X_test).float()
y_test = torch.tensor(y_test).float()

for type_name, df, is_gru, is_cut in [('LSTM', lstm_hist, False, False), ('GRU', gru_hist, True, False), ('LSTM cut', lstm_cut_hist, False, True), ('GRU cut', gru_cut_hist, True, True)]:
  hists[type_name] = {}
  for index, row in df.iterrows():
    num_hiddens, lr, num_outs = row['num hiddens'], row['lr'], row['out layers']
    net_str = net_to_str(num_hiddens, lr, num_outs)
    net = net_from_row(num_hiddens, lr, num_outs, is_gru)
    net.load_state_dict(torch.load(path_to_net(num_hiddens, lr, num_outs, is_gru, is_cut), map_location=torch.device('cpu')))
    y_pred = net(X_test)
    print(y_pred.item())
    hists[type_name][net_str] = (y_test.item() - y_pred.item())**2

0.327607125043869
0.09042935818433762
0.3693057894706726
0.3357864320278168
0.3727995455265045
0.3809801936149597
0.5240792632102966
0.33944380283355713
0.9371748566627502
0.40904369950294495
0.18345268070697784
0.32436859607696533
0.8598317503929138
0.29066938161849976
0.34819552302360535
0.0859159454703331
0.3762205243110657
0.4789056181907654
0.7530215978622437
0.09049852937459946
0.5423709750175476
0.5650705695152283
0.47488653659820557
0.3973510265350342
1.2122842073440552
0.09046674519777298
0.5386286973953247
0.36713773012161255
0.5873066186904907
0.44007396697998047
0.7547754049301147
0.36637160181999207
0.46691906452178955
0.41340547800064087
0.4869762659072876
0.5076968669891357
0.7639226913452148
0.719633936882019
0.3222196698188782
0.465771347284317
0.5340539216995239
0.3844190835952759
1.0817385911941528
0.09053947776556015
0.5529173016548157
0.735509991645813
0.5934807658195496
0.5024811625480652
0.8187223672866821
0.0963655486702919
0.9148222208023071
0.602830708026886
0

In [53]:
for k, v in hists.items():
  print(f'{k} min loss is {min(list(v.values()))}, argmin is {min(v, key=v.get)}')

LSTM min loss is 3.416867439742788e-05, argmin is 400-0.0008-[400]
GRU min loss is 2.6991655958319427e-05, argmin is 400-0.0005-[400]
LSTM cut min loss is 2.5254900180016193e-06, argmin is 400-0.001-None
GRU cut min loss is 4.629264971356406e-09, argmin is 200-0.001-None
